In [1]:
using CSV
using DataFrames
using XLSX
using Statistics
using Glob

In [2]:
#=
This function removes the unnecessary columns for the input of BNN
This function also writes the processed data into its respective CSV file
=#
function process(otu, res, otu_idx, res_idx, level)
    # join the otus and responses by sample ID
    data = innerjoin(otu, res, on = :Column1)
    # find the total number of columns
    col_length = size(data)[2]
    # remove the "variety2" and real-valued response column
    data = data[:, Not((col_length-2):(col_length-1))]
    # remove the sample ID
    data = data[:, Not(1)]
    # write the data to a CSV file with its specified name
    mat = Matrix(data)
    filename = string(otu_idx, "_", res_idx)
    CSV.write("../processed-data/$level/full-data/$filename.csv", Tables.table(mat), header=false)
end

process (generic function with 1 method)

In [3]:
#= 
This function load the filtered otu and responses 
=#
function load(level)
    # load all filtered data in one level folder
    otu_path = "../processed-data/filter-data/$level"
    otu_files = glob("*.csv", otu_path);
    otu = DataFrame.(CSV.File.(otu_files));
    
    # load all responses
    response_path = raw"../processed-data/response"
    response_files = glob("*.csv", response_path)
    response = DataFrame.(CSV.File.(response_files));
    
    # pass them to process and write to new CSVs
    for i in 1:length(otu)
        for j in 1:length(response)
            process(otu[i], response[j], i, j, level)
        end
    end
end

load (generic function with 1 method)

In [4]:
load("Phylum")